In [1]:
import sys
sys.path.append('/data/users/fyx/NTMC-Community/MatchZoo-py/')
import matchzoo as mz

In [2]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [3]:
preprocessor = mz.models.DUET.get_default_preprocessor(
    filter_mode='df',
    filter_low_freq=2,
    truncated_mode='post',
    truncated_length_left=10,
    truncated_length_right=40,
    ngram_size=3
)

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6447.42it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3617.47it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1976372/1976372 [00:00<00:00, 2799741.35it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7512.74it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3447.10it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 688621.98it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7693.66it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [5]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x7fbf130aa908>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fbf130aaac8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fc01b431c88>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fbf1036f5c0>,
 'ngram_vocab_size': 6602}

In [6]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    callbacks=[triletter_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[triletter_callback]
)

In [7]:
padding_callback = mz.models.DUET.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=40,
    pad_word_value=0,
    pad_word_mode='pre',
    with_ngram=True
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.DUET()

model.params['task'] = ranking_task
model.params['left_length'] = 10
model.params['right_length'] = 40
model.params['lm_filters'] = 100
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 100
model.params['mlp_activation_func'] = 'tanh'

model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['dm_conv_activation_func'] = 'relu'
model.params['dm_filters'] = 100
model.params['dm_kernel_size'] = 3
model.params['dm_right_pool_size'] = 4
model.params['dropout_rate'] = 0.2


model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DUET(
  (lm_conv1d): Conv1d(40, 100, kernel_size=(1,), stride=(1,))
  (lm_mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1000, out_features=100, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
    )
    (2): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
    )
  )
  (lm_linear): Sequential(
    (0): Linear(in_features=100, out_features=1, bias=True)
    (1): ReLU()
  )
  (dm_conv_activation_func): ReLU()
  (dm_conv_left): Conv1d(6602, 100, kernel_size=(3,), stride=(1,))
  (dm_mlp_left): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
  )
  (dm_conv1_right): Conv1d(6602, 100, kernel_size=(3,), stride=(1,))
  (dm_conv2_right): Conv1d(100, 100, kernel_size=(1,), stride=(1,))
  (dm_mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=900, out_features=100, bias=True)
      (1)

In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-102 Loss-0.992]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2579 - normalized_discounted_cumulative_gain@5(0.0): 0.3463 - mean_average_precision(0.0): 0.32

